# Pairs Trading Strategy Design & Backtest

The definition of Pairs Trading is to take advantage of the mispricing between two or more assets by taking long and short portfolio, betting the related movement will converge back when mispricing situation happening. The principle is buy undervalued and sell overvalued. This is a very charming strategy that we can extend the idea in any market of interest. 

The first pairs trading bears its roots in equity market, but we can possibly extend the idea into commodities, cryptos, and even options market.

### Import required lib

In [ ]:
import warnings
import numpy as np
import pandas as pd
import datetime as dt
import statsmodels.api as sm
from statsmodels.tsa.api import adfuller
import matplotlib.pyplot as plt
import pyfolio as pf
import yfinance as yf
from scipy import optimize
from scipy.stats import norm
warnings.filterwarnings("ignore")

The numerical techniques are regression computation in matrix form, Engle-Granger procedure, and statistical tests.

A) multivariate cointegration (Johansen Procedure)


B) robustness checking of cointegration weights, ie, by adaptive estimation of your regression parameters with statistical filters.

## Signal Generation and Backtesting
- Be inventive beyond equity pairs: consider commodity futures, instruments on interest rates, and aggregated indices. (Initially consider to find the candidate pairs from crude oil)
- Arb is relized by using cointegrating coff as allocation w. all project designs should including trading signal generation(OU process fitting) and backtesting.
- does P&L behave as expected for cointegration arb trade? Is P&L coming from a few or many trades, what is half-life? Maximum Drawdown and behaviour of volatility/VaR?
- Introduce liquidity and algorithmic flow considerations (a model of order flow). Any rules on accumulating the position? What impact bid-ask spread and transaction costs will make?
## Step-by-step instructions
- multivariate cointegration to identify cases
- Part I: pairs trading design
    - 1. re-code regression estimation in martrix form - your own OLS implementation which you can re-use. Regression between stationary variables(DF test regression/difference equation) has optional model specification test for (a)identifying optimal lag p with AIC BIC tests and (b) stability check
    - 2. Implement Engle-Granger procedure for each pair. Step1 use ADF test for unit root with lag1. Step2, formulate both correction equation and decide which one is more important
    - 3. Decide signals: $\mu_e +- Z\sigma_{eq}$ and exit on $\mu_t$
    - 4. At first aussme Z=1. Then change Z sightly upwards and downwards - compute P&L for each case of bounds. Alternatively run an optimization that varies Z and maxmize the P&L or other criterion.
    - 5. Optionally us VECM in order select the best candidate for pairs trading (or basket trading).
    
- Part II: Backtesting
    - 1. perform systematic backtesting of your trading strategy platform o produce drawdown plots, rolling Sharpe ratio and rolling beta
    - 2. keep delivering staionary spread over 3-6 months. Kalman filter will give updated beta. However, you can simply re-estmate cointergration by shifting data 1-2 weeks and report beta and EG.
    - 3. use time series CV
    
    
Time Series Project Workshop, Cointegration Lecture and Pairs Trading tutorial are your key resources.

# Part I: Pairs Trading Design
## Data Processing
One study by Jacob & Weber conducted several international markets which has empirically proven that the pairs trading works the most in emerging market, either from the high ineffciencies or a large number of available pairs. So I believe some innovative market has more opportunities than equity market.

In this case, I want to study model-driven statistical arbitrage strategies in U.S. equities, commodities and crypto market. The crpyto is the youngest and has less research than other assets, which becomes very attractive for pairs trading strategy design. From those three different perspectives, we can identify multiple strategy implementations and more profitable opportunities. So, in the first step, we sort out a list of available symbols and prepare them for filtering.

In [ ]:
## In this case, I want to implement pairs trading from two perspectives - equity and commodity market.
## First I listed some potentially profitable tickers to be tested from different market.
start = '2005-01-01'
end = '2022-07-30'
tickers_commodity = {'Gold':'GC=F',
                     'Sliver':'SI=F',
                     'Crude Oil':'CL=F',
                     'Natural Gas':'NG=F',
                     'Gasoline':'RB=F',
                     'E-Mini S&P 500':'ES=F'}

tickers_crypto = ['BTC-USD','ETH-USD','ADA-USD','SOL-USD','BNB-USD','DOGE-USD','LTC-USD']

tickers_stock = ['']
price_commodity = yf.download(list(tickers_commodity.values()), start, end)['Adj Close'].dropna()
price_commodity.rename({v:k for k,v in tickers_commodity.items()},axis=1,inplace=True)

price_crypto = yf.download(tickers_crypto, '2017-12-01', end)['Adj Close'].dropna()['2021-07':]
price_crypto.head()

In [ ]:
price_commodity.head()

## Cointegration Apporach
Cointegration: I(d) series, which means integrated series of order d
I(1) series: Price
I(0) series: Returns
The prices of cointegrated assets fluctuate around a certain average level. So cointegration allows us to construct a 2-asset portfolio with stationary series to be traded. Then we are able to construct a mean-reversion strategy.

### Find $\beta_{Coint}$
- Engle-Grange test
    - Linear regression on the candidate pairs price and calculate its residual
    - Test the stationary of the residual
- Johansen test
    - VECM
    
    
We have two apporaches to find cointegration beta parameter. 
##### Engle-Grange test
The first idea of Engle Granger test is performing a linear regression between two underlying assets and test its residual and see if the series is stationary by applying Augmented Dick-Fuller test. So if the residual is a stationary series, we can say the two prices are cointegrated. The $\beta_{Coint}$ is obtained as the asset weight to be traded.

In the stationarity test, we test for a unit root, which is based on the following hypothesis test:
$$H_0: \phi =1 \rightarrow y_t \sim I(0) | (unit root)$$
$$H_1:\mid\phi\mid <1 \rightarrow y_t \sim I(0) | (stationary)$$

## Pair Candidates Selection
Before we do cointegration, we have 2 baskets of asset - commodity and crypto. We will only look for cointegrated pairs inside the basket because ADF test is not good at identifying spurious relationships.

A

In [ ]:
def pairs_selection(prices):
    '''
    :param prices: dataframe-like, all asset prices to be checked to find candidate pairs
    '''
    res = pd.DataFrame(columns = ['p-value','aic'])
    n = prices.shape[1]
    for i in range(n):
        for j in range(i+1, n):
            pairs = prices.iloc[:,[i,j]]
#             print(pairs)
            pairs_name = f"({pairs.columns[0]}, {pairs.columns[1]})"
            y = pairs.iloc[:,0]
            x = pairs.iloc[:,1]
            beta, resid, summary = OLS(y, x)
            
            summary, p, aic = ADF_test(resid, pairs_name, verbose=False)
            res.loc[pairs_name] = [p, aic]
    res.sort_values(by=['p-value'],inplace=True)
    return res


### Filtering of Commodity Basket
By calculating the ADF p-values for all pairs, we find that crude oil and gasoline have the most significant cointegration relationship, which is highly reasonable because the properties of these two commodities are very close, with crude oil being the raw material for gasoline.

So in the commodity basekt, we will further study this candidate pair.

In [ ]:
pairs_selection(price_commodity).head()

In [ ]:
y_commodity = price_commodity.loc[:,'Crude Oil']
x_commodity = price_commodity.loc[:,'Gasoline']

beta_commodity, resid_commodity, summary_commodity = OLS(y_commodity, x_commodity)
# resid.plot(figsize=(10,8))
fig, ax = plt.subplots(1,1,figsize=(8,6))
ax.plot(resid_commodity)
ax.set_title('Res(Crude Oil, Gasoline)')
ax.set_xlabel('Date')


### Filtering of Crypto Basket
The cryptocurrency market is very volatile and we can't ensure that the pairs have all time cointegration, so we are looking forward a year to find the relationship over that time.

Fortunately, we found that the BNB-ETH pair was a possible integrated pair in the previous year, so we can assume that this relationship will be maintained for a short time in the following.

In [ ]:
pairs_selection(price_crypto).head()

In [ ]:
y_crypto = price_crypto.loc[:,'BNB-USD']
x_crypto = price_crypto.loc[:,'ETH-USD']

beta_crypto, resid_crypto, summary_crypto = OLS(y_crypto, x_crypto)
# resid.plot(figsize=(10,8))
fig, ax = plt.subplots(1,1,figsize=(8,6))
ax.plot(resid_crypto)
ax.set_title('Res(BNB, ETH)')
ax.set_xlabel('Date')

### ENGLE-GRANGER STEP 1. Cointegrated Residual

In [ ]:

def OLS(y, x):
    '''
    parameters:
    :param y: independent variable, dataframe or array-like
    :param x: dependent variables, dataframe or array-like
    :return:
    '''


    model = sm.OLS(y, sm.add_constant(x)).fit()

    residuals = model.resid
    residuals = pd.DataFrame({'resid':residuals},index = x.index)
    ## OLS params
    c, beta = model.params

    ## OLS params sd
    c_sd, beta_sd = model.bse

    # OLS t-statistics
    c_t, beta_t = model.tvalues
    
    
    summary = pd.DataFrame({"Params":model.params,
                       "Error":model.bse,
                       'T-stats':model.tvalues,
                       'P-values': model.pvalues})
    
    return beta, residuals, np.around(summary,2)




### ADF test
**ADF test equation**
use ADF test for unit root with lag1:
$$\Delta e_t = \varphi e_{t-1} + \varphi_{aug1}\Delta e_{t-1} + const + \varphi_tt+\epsilon_t$$
- Improvement 1. Test equation above includes time dependence  $\varphi_tt$ , referred to as 'trend'.
I don't include trend in the ADF tests and cointegrating residual -- it will make me think cointegration is present when it is very weak. In fact, without $\varphi_tt$ term, we might not even get stationarity result.

In [ ]:
def ADF_test(resid, name, verbose=True):
    # H_0 in adfuller is unit root exists (non-stationary)
    # We must observe significant p-value to convince ourselves that the series is stationary
    '''
    :param resid: dataframe-like, the residual from OLS or any other series to be tested
    '''
    index = resid.index
    resid = np.array(resid).flatten()
    series = pd.DataFrame({'e_t':resid},index = index)
    series['e_t-1'] = series['e_t'].shift(1)
    series['Δe_t'] = series['e_t'].diff()
    series['Δe_t-1'] = series['e_t'].diff().shift(1)
    
    series = series.dropna()
    x = series[['e_t-1','Δe_t-1']]
    y = series['Δe_t']
    
    model = sm.OLS(y, sm.add_constant(x)).fit()

    summary = pd.DataFrame({f"Estimate ΔRes({name})":model.params,
                           f"SD of Estimate ΔRes({name})":model.bse,
                           f"t-Statistic ΔRes({name})":model.tvalues,
                           f"P-value ΔRes({name})":model.pvalues})
    summary = np.around(summary,6)
#     display(summary)

    adf = adfuller(series['e_t'],regression='c')
    aic = adf[-1]

    pvalue = round(adf[1],6)
    if verbose==True:
        print("---------ADF result--------:")
        if pvalue < 0.05:
            print('p-value = ' + str(pvalue) + ' The series ' + name +' is likely stationary.')
        else:
            print('p-value = ' + str(pvalue) + ' The series ' + name +' is likely non-stationary.')
    
    return summary, pvalue, aic

summary, p, aic = ADF_test(resid_commodity, 'Crude Oil, Gasoline')
display(summary)
summary, p, aic = ADF_test(resid_crypto, 'BNB, ETH') 
display(summary)

Both pairs passed the ADF test.

In [ ]:
### ENGLE-GRANGER STEP 2. Error correction equations
In general form, 
$$\Delta P_t^A = \varphi \Delta P_t^B - (1-\alpha) \tilde e^A_{t-1} + \epsilon_t$$

In the other way around:
$$\Delta P_t^B = \varphi \Delta P_t^A - (1-\alpha) \tilde e^A_{t-1} + \epsilon_t$$

In [ ]:
def EG_err_corr1(S1, S2, e, name):
    '''
    :param S1: dataframe-like, the price of asset A
    :param S2: dataframe-like, the price of asset B
    :param e: dataframe-like, the residual between A and B
    '''
    dS1 = np.array(S1.diff()).flatten()
    dS2 = np.array(S2.diff()).flatten()
    
    e = np.array(e.shift(1)).flatten()
    df_to_fit = pd.DataFrame({'ΔPA_t':dS1,
                             'ΔPB_t':dS2,
                             'e_t-1':e}).dropna()
    
    x = df_to_fit[['ΔPB_t','e_t-1']]
    y = df_to_fit['ΔPA_t']
    
    model = sm.OLS(y, x).fit()

    summary = pd.DataFrame({f"Estimate Δ{name}":model.params,
                           f"SD of Estimate Δ{name}":model.bse,
                           f"t-Statistic Δ{name}":model.tvalues,
                           f"P-value Δ{name}":model.pvalues})
    summary = np.round(summary,6)
    return summary, model.pvalues
summary, p = EG_err_corr1(y_commodity, x_commodity, resid_commodity, 'Gasoline')
display(summary)
summary, p = EG_err_corr1(y_crypto, x_crypto, resid_crypto, 'ETH')
display(summary)

We now perform **the 2nd step of EG to estimate the Equilibrium Correction Model**, Here we check the significance of -(1-𝛼) term, which ensure the correction the long run equilibrium. In error correction equation the p-value is significantly showing 0.

In [ ]:
def EG_err_corr2(S1, S2, e, name):
    '''
    :param S1: dataframe-like, the price of asset A
    :param S2: dataframe-like, the price of asset B
    :param e: dataframe-like, the residual between A and B
    '''
    dS1 = np.array(S1.diff()).flatten()
    dS2 = np.array(S2.diff()).flatten()
    
    e = np.array(e.shift(1)).flatten()
    df_to_fit = pd.DataFrame({'ΔPA_t':dS1,
                             'ΔPB_t':dS2,
                             'e_t-1':e}).dropna()
    
    x = df_to_fit[['ΔPA_t','e_t-1']]
    y = df_to_fit['ΔPB_t']
    
    model = sm.OLS(y, x).fit()

    summary = pd.DataFrame({f"Estimate Δ{name}":model.params,
                           f"SD of Estimate Δ{name}":model.bse,
                           f"t-Statistic Δ{name}":model.tvalues,
                           f"P-value Δ{name}":model.pvalues})
    summary = np.round(summary,6)
    
    return summary, model.pvalues
summary, p = EG_err_corr2(y_commodity, x_commodity, resid_commodity, 'Crude Oil')
display(summary)
summary, p = EG_err_corr2(y_crypto, x_crypto, resid_crypto, 'BNB')
display(summary)

Now, we estimate the EG correction equation "other way around", both shows the significance.
From the absolute value of t-stas, the first way is the more important model. So the pair is considered to be cointegrated.


## Ornstein-Uhlenbeck process

In order to find the optimal $\beta_{Coint}$ to build the best mean reversion portfolio, we can fit the OU process.
 
The Ornstein-Uhlenbeck process is described by the following SDE:

$$ dX_t = \kappa (\theta - X_t) dt + \sigma dW_t $$

The parameters are:
- $\theta \in \mathbb{R}$: The long-term average, around which all trajectories of $X_t$ oscillate, is the mean level.

- $\kappa > 0$: the speed of mean reversion, represents the velocity at which such trajectories will regroup around mean level

- $\sigma > 0$: instantaneous volatility, measures the amplitude of randomness entering the system.


At this point we can solve the SDE:

$$ X_t = \theta + (X_0 - \theta)e^{-\kappa t} + \int_0^t \sigma\, e^{\kappa (s-t)} dW_s .$$


### Moments:

The **mean** of $X_t$ is:

$$ \begin{aligned}
\mathbb{E}[X_t] &= \mathbb{E}\biggl[ \theta + (X_0 - \theta)e^{-\kappa t} + \int_0^t \sigma\, e^{\kappa (s-t)} dW_s \biggr] \\
                &= \theta + (X_0 - \theta)e^{-\kappa t}    
\end{aligned}$$

The **covariance** is:

$$ \begin{aligned}
\text{Cov}[X_s, X_t] &= \frac{\sigma^2}{2\kappa} \biggl( e^{-\kappa |t-s|} - e^{-\kappa (s+t)}\, \biggr),  
\end{aligned}$$

The **variance** is: 

$$ \text{Var}[X_t] = \text{Cov}[X_t, X_t] = \frac{\sigma^2}{2\kappa} \biggl( 1- e^{-2 \kappa t} \biggr).$$

So, we can obtain the **mean**: $\theta$ and the **variance**: $\frac{\sigma^2}{2\kappa}$




We can discretize the SDE using the Euler-Maruyama numerical method:

Let us consider the solution of the OU SDE obtained above. We can compute $X_{n+1}$ and consider the initial value at time $n$.

$$ X_{n+1} = \theta + (X_n - \theta)e^{-\kappa \Delta t} + \sqrt{\frac{\sigma^2}{2\kappa} \bigl( 1- e^{-2 \kappa \Delta t} \bigr)} \; \epsilon_n $$ 

with $\epsilon_n \sim \mathcal{N}(0,1)$.



## Estimation of parameters

We can compute $X_{t+\Delta t}$ and consider the initial value at time $t$.

$$ \begin{aligned}
X_{t+\Delta t} &= \theta + (X_t - \theta)e^{-\kappa \Delta t} + \int_t^{t+\Delta t} \sigma\, e^{\kappa (s-t)} dW_s \\
               &= \theta \bigl( 1-e^{-\kappa \Delta t} \bigr) + e^{-\kappa \Delta t} X_t + \int_t^{t+\Delta t} \sigma\, e^{\kappa (s-t)} dW_s \\
               &= \alpha + \beta X_t + \epsilon_t
\end{aligned} $$

where $\alpha = \theta \bigl( 1-e^{-\kappa \Delta t} \bigr)$, $\beta = e^{-\kappa \Delta t}$ and with $\epsilon_t \sim \mathcal{N}\biggl( 0, \frac{\sigma^2}{2\kappa} \bigl( 1- e^{-2 \kappa \Delta t} \bigr)\biggr)$.

So, this confirms the saying from "The Ornstein–Uhlenbeck process can also be considered as the continuous-time analogue of the discrete-time AR(1) process." and we are able to guarantee the AR(1) process to estimate the params on the spread. 

let us use the usual OLS method to estimate $\alpha$, $\beta$ and $\sigma$.
Then, we can obtain the parameters from the formulas:

$$ \kappa = - \frac{\log \beta}{\Delta t}, \quad \theta = \frac{\alpha}{1-\beta}, \quad 
\sigma = \text{Std}[\epsilon_t] \sqrt{ \frac{2\kappa}{1-\beta^2} }$$

we can obtain almost consistent parameters to those params obtained by MLE.

Halflife:
$$Halflife(days) = \frac{ln(2)}{\theta*dt}$$

In [ ]:
resid

In [ ]:
class OU_process:
    def __init__(self, freq = 'D'):
        if freq =='D':
            self.dt = 1/252
        elif freq=='H':
            self.dt = 1/252/60
        elif freq=="M":
            self.dt = 1/252/60/60
        
        
    
    def fit(self,resid, verbose = True):
        X = np.array(resid[:-1]).flatten()
        Y = np.array(resid[1:]).flatten()

        model = sm.OLS(Y, sm.add_constant(X)).fit()
        alpha, beta = model.params
        kappa = - np.log(beta)/self.dt
        theta = alpha/(1-beta)
        res = Y - beta * X - alpha                   # residuals
        std_resid = np.std(res, ddof=2)
        sigma = std_resid * np.sqrt(2*kappa/(1-beta**2))
        sigma_eq = std_resid*np.sqrt(1/(1-beta**2))
        halflife = np.log(2)/kappa/dt
        if verbose:
            print("OU process params:")
            print(f"theta = {theta:.4f}")
            print(f"kappa = {kappa:.4f}")
            print(f"sigma = {sigma:.4f}")
            print(f"sigma_eq = {sigma_eq:.4f}")
            print(f"halflife = {halflife:.4f}")
        self.params = {'theta':theta,'kappa':kappa, 'sigma':sigma,'sigma_eq':sigma_eq,'halflife':halflife}
        return self



### Commodity basket OU process fitting


In [ ]:
def OU_process_pairs_selection(prices, kappa_thres = 5):
    res = pd.DataFrame(columns = ['theta','kappa', 'sigma','sigma_eq','halflife'])
    resid_df = pd.DataFrame(index = prices.index)
    n = prices.shape[1]
    for i in range(n):
        for j in range(i+1, n):
            pairs = prices.iloc[:,[i,j]]
            pairs_name = f"({pairs.columns[0]}, {pairs.columns[1]})"
            y = pairs.iloc[:,0]
            x = pairs.iloc[:,1]
            beta, resid, summary = OLS(y, x)
            
            params = OU_process().fit(resid,verbose=False).params
            if params['kappa']>kappa_thres:
                res.loc[pairs_name] = list(params.values())
                resid_df[pairs_name] = resid
    res.sort_values(by=['kappa'],ascending=False,inplace=True)
    return res, resid_df


In [ ]:
res, resid_df = OU_process_pairs_selection(price_commodity)
res

In [ ]:
res, resid_df = OU_process_pairs_selection(price_crypto, kappa_thres=8)
res

OU process pairs selection

解释一下继续选择那两个pair是合理的

# Signal Generation

# Source:
[1] https://www.sciencedirect.com/science/article/abs/pii/S1386418114000809

[2] https://github.com/cantaro86/Financial-Models-Numerical-Methods/blob/master/6.1%20Ornstein-Uhlenbeck%20process%20and%20applications.ipynb

# Signal Generation

# Source:
[1] https://www.sciencedirect.com/science/article/abs/pii/S1386418114000809

[2] https://github.com/cantaro86/Financial-Models-Numerical-Methods/blob/master/6.1%20Ornstein-Uhlenbeck%20process%20and%20applications.ipynb

In [507]:
price_commodity.iloc[:,[1,3]]

,E-Mini S&P 500,Natural Gas
Date,,
2005-01-03,1206.25,5.790
2005-01-04,1191.00,5.902
2005-01-05,1183.25,5.833
2005-01-06,1188.25,6.049
2005-01-07,1186.25,6.001
...,...,...
2022-07-25,3970.00,8.727
2022-07-26,3923.25,8.993
2022-07-27,4024.50,8.687


In [508]:
adfuller?